In [ ]:
from src.all_in_one import *

In [ ]:
ref = pd.read_excel(r'D:\fc-performeter\notebooks\referrals.xlsx', engine='openpyxl')

In [ ]:
# Mapping referral directions
ref.insert(6, 'ext_ref', ref['OutgoingInternalIngoing'].apply(lambda x: 'Yes' if x == 'outgoing' else 'No' )) # cases referred from ADMSP to Others
ref.insert(7, 'int_ref', ref['OutgoingInternalIngoing'].apply(lambda x: 'Yes' if x == 'internal' else 'No' )) # cases internally referred
ref.insert(8, 'rec_ref', ref['OutgoingInternalIngoing'].apply(lambda x: 'Yes' if x == 'ingoing' else 'No' ))  # cases received from others

In [ ]:
# Choose what columns are not needed here
ref = ref.drop(columns=['age', 'country', 'personal/familyMember', 'familyMemberNumber', 'group/individual', 'OutgoingInternalIngoing', 'note'])

In [ ]:
# columns renaming
ref.rename(columns={
    'referral_ID': 'rid',
    'FC_ID': 'fcid',
    'beneficiaryName': 'ben_name',
    'gender': 'sex',
    'socialAssessmentDate': 'social_ass_date',
    'referralDate': 'ref_date',
    'referredToFromEntity': 'entity',
}, inplace=True)

In [ ]:
# Type transformation
ref['fcid'] = ref['fcid'].astype("Int64")
ref['ref_date'] = pd.to_datetime(ref['ref_date'])

In [ ]:
ref.head()

In [ ]:
ref['entity'].value_counts()

In [ ]:
# separating dataframe upon referral types
ext_ref = ref[ref['ext_ref'] == 'Yes'] 

In [ ]:
ext_ref = ext_ref.drop(columns=['int_ref', 'rec_ref'])

In [ ]:
ext_ref